In [ ]:
import glob

from potoo.plot import *
from potoo.util import *
import sklearn

from cache import *
from datasets import *
from features import *
from load import *
from metadata import *
from sp14.model import *
from util import *
from viz import *

figsize('inline_short');

In [ ]:
xc_counts = (pd.read_csv(f'{metadata_dir}/xeno-canto-rec-count-per-species.csv')
    # Lowercase col names so we don't break when we e.g. twiddle casing in the spreadsheet
    .pipe(df_transform_column_names, lambda c: c.lower())
    .drop(columns=[
        'unnamed: 5',
    ])
    .rename(columns={
        'common name': 'com_name',
        'scientific name': 'sci_name',
        'extinct?': 'extinct',
        '# recs': 'n_recs',
        '# bg recs': 'n_bg_recs',
        'species index': 'species_index',
    })
)
display(
    df_summary(xc_counts),
    xc_counts[:10],
    df_summary(metadata.species.df),
    metadata.species.df[-10:],
)

,com_name,sci_name,extinct,n_recs,n_bg_recs,species_index
dtype,object,object,object,int64,int64,int64
sizeof,791.7 KiB,797.0 KiB,339.5 KiB,373.2 KiB,359.3 KiB,376.7 KiB
len,10711,10711,10711,10711,10711,10711
count,10711,10711,149,10711,10711,10711
nunique,10711,10711,1,395,429,10711
mean,NaN,NaN,NaN,36.381,30.383,5356.000
std,NaN,NaN,NaN,100.879,199.042,3092.144
min,Abbott's Babbler,Abeillia abeillei,Extinct,0,0,1
25%,Crested Bunting,Cotinga maynana,Extinct,4,0,2678
50%,Mangrove Finch,Loriculus vernalis,Extinct,13,1,5356


,com_name,sci_name,extinct,n_recs,n_bg_recs,species_index
0,Great Tit,Parus major,NaN,3726,5020,1
1,Common Chaffinch,Fringilla coelebs,NaN,2762,8799,2
2,Common Blackbird,Turdus merula,NaN,2638,8341,3
3,Common Chiffchaff,Phylloscopus collybita,NaN,2508,2827,4
4,Eurasian Blue Tit,Cyanistes caeruleus,NaN,1893,3404,5
5,European Robin,Erithacus rubecula,NaN,1853,5063,6
6,Red Crossbill,Loxia curvirostra,NaN,1825,583,7
7,Eurasian Blackcap,Sylvia atricapilla,NaN,1793,3505,8
8,Willow Warbler,Phylloscopus trochilus,NaN,1738,1801,9
9,Song Thrush,Turdus philomelos,NaN,1708,3841,10


,shorthand,longhand,sci_name,com_name,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes
dtype,category,category,category,category,category,category,object,object,object,object
sizeof,636.2 KiB,859.3 KiB,785.3 KiB,779.8 KiB,669.9 KiB,658.5 KiB,660.5 KiB,597.3 KiB,628.7 KiB,390.6 KiB
len,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552
count,10552,10552,10552,10552,10552,10552,10552,9038,10552,2144
nunique,10552,10552,10552,10552,10552,10552,10552,7090,6250,2144
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,COOS,Common Ostrich - COOS,Struthio camelus,Common Ostrich,TC000002,ostric2,10000.0,AAKS,ABAB,ABDU
25%,STSP,Sao Tome Spinetail - STSP,Zoonavena thomensis,Sao Tome Spinetail,TC003784,satspi1,15963.0,CRCH CARC,COSQ,CPOT
50%,VEBT,Velvety Black-Tyrant - VEBT,Knipolegus nigerrimus,Velvety Black-Tyrant,TC006272,vebtyr1,23377.01045,MARI,LOSU,LIBI


,shorthand,longhand,sci_name,com_name,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes
10542,JPIN,Jos Plateau Indigobird - JPIN,Vidua maryae,Jos Plateau Indigobird,TC012204,jopind1,32168.0,JPIN,VIMA,NaN
10543,JAIN,Jambandu Indigobird - JAIN,Vidua raricola,Jambandu Indigobird,TC012205,jamind1,32169.0,JAIN,VIRA,NaN
10544,BAIN,Baka Indigobird - BAIN,Vidua larvaticola,Baka Indigobird,TC012206,bakind1,32170.0,BAIN,VILA,NaN
10545,CAIN,Cameroon Indigobird - CAIN,Vidua camerunensis,Cameroon Indigobird,TC012207,camind1,32171.0,CAIN,VICA,NaN
10546,VAIN,Variable Indigobird - VAIN,Vidua funerea,Variable Indigobird,TC012208,varind1,32172.0,VAIN DUIN,VIFU,NaN
10547,PUIN,Purple Indigobird - PUIN,Vidua purpurascens,Purple Indigobird,TC012209,purind1,32175.0,PUIN,VIPU,NaN
10548,GRIN,Green Indigobird - GRIN,Vidua codringtoni,Green Indigobird,TC012210,greind1,32176.0,GRIN ZAIN,VICO,NaN
10549,PAWE,Parasitic Weaver - PAWE,Anomalospiza imberbis,Parasitic Weaver,TC012212,parwea1,32178.0,PAWE CUFI,ANIM,NaN
10550,_UNK,Unknown - _UNK,Unknown,Unknown,TC___UNK,___unk,_UNK,_UNK,_UNK,_UNK
10551,_NON,No species - _NON,No species,No species,TC___NON,___non,_NON,_NON,_NON,_NON


In [ ]:
outer_join = pd.merge(
    metadata.species.df.assign(ebird=True),
    xc_counts.assign(xc=True),
    how='outer',
    on='sci_name',
)
display(
    df_summary(outer_join),
    outer_join[-10:],
)

,shorthand,longhand,sci_name,com_name_x,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,ebird,com_name_y,extinct,n_recs,n_bg_recs,species_index,xc
dtype,category,category,object,category,category,category,object,object,object,object,object,object,object,float64,float64,float64,object
sizeof,754.5 KiB,978.2 KiB,943.3 KiB,898.4 KiB,788.2 KiB,776.8 KiB,778.6 KiB,715.7 KiB,746.8 KiB,508.6 KiB,489.0 KiB,905.5 KiB,457.1 KiB,447.8 KiB,447.8 KiB,447.8 KiB,489.6 KiB
len,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463,11463
count,10552,10552,11463,10552,10552,10552,10552,9038,10552,2144,10552,10711,149,10711,10711,10711,10711
nunique,10552,10552,11463,10552,10552,10552,10552,7090,6250,2144,1,10711,1,395,429,10711,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000,NaN,NaN,36.381,30.383,5356.000,1.000
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,100.879,199.042,3092.144,0.000
min,_NON,No species - _NON,Abeillia abeillei,No species,TC___NON,___non,10000.0,AAKS,ABAB,ABDU,True,Abbott's Babbler,Extinct,0.0,0.0,1.0,True
25%,VICP,Victoria Crowned-Pigeon - VICP,Cotinga amabilis,Victoria Crowned-Pigeon,TC002632,vicpig1,15963.0,CRCH CARC,COSQ,CPOT,True,Crested Bunting,Extinct,4.0,0.0,2680.0,True
50%,BCFG,Black-capped Foliage-gleaner - BCFG,Loxioides bailleui,Black-capped Foliage-gleaner,TC005394,bcfgle1,23377.01045,MARI,LOSU,LIBI,True,Mangrove Finch,Extinct,13.0,1.0,5360.0,True


,shorthand,longhand,sci_name,com_name_x,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,ebird,com_name_y,extinct,n_recs,n_bg_recs,species_index,xc
11453,nan,nan,Pyrgilauda theresae,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Afghan Snowfinch,NaN,0.0,0.0,10600.0,True
11454,nan,nan,Foudia aldabrana,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Aldabra Fody,NaN,0.0,0.0,10600.0,True
11455,nan,nan,Coccopygia bocagei,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Angolan Waxbill,NaN,0.0,0.0,10600.0,True
11456,nan,nan,Lonchura leucosticta,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,White-spotted Mannikin,NaN,0.0,0.0,10600.0,True
11457,nan,nan,Macronyx sharpei,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Sharpe's Longclaw,NaN,0.0,0.0,10700.0,True
11458,nan,nan,Leucosticte sillemi,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Sillem's Mountain Finch,NaN,0.0,0.0,10700.0,True
11459,nan,nan,Crithagra canicapilla,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,West African Seedeater,NaN,0.0,0.0,10700.0,True
11460,nan,nan,Spinus cucullata,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Red Siskin,NaN,0.0,0.0,10700.0,True
11461,nan,nan,Pipilo naufragus,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Bermuda Towhee,Extinct,0.0,0.0,10700.0,True
11462,nan,nan,Hemispingus goeringi,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Slaty-backed Hemispingus,NaN,0.0,0.0,10700.0,True


In [ ]:
# ebird only:
#   - 752
#   - ?
ebird_only = outer_join[lambda df: pd.notnull(df.ebird) & pd.isnull(df.xc)]
display(
    df_summary(ebird_only),
    ebird_only[-10:],
)

,shorthand,longhand,sci_name,com_name_x,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,ebird,com_name_y,extinct,n_recs,n_bg_recs,species_index,xc
dtype,category,category,object,category,category,category,object,object,object,object,object,object,object,float64,float64,float64,object
sizeof,51.3 KiB,67.6 KiB,62.3 KiB,61.9 KiB,53.7 KiB,52.9 KiB,53.4 KiB,49.7 KiB,50.8 KiB,32.8 KiB,32.3 KiB,29.4 KiB,29.4 KiB,29.4 KiB,29.4 KiB,29.4 KiB,29.4 KiB
len,752,752,752,752,752,752,752,752,752,752,752,752,752,752,752,752,752
count,752,752,752,752,752,752,752,673,752,117,752,0,0,0,0,0,0
nunique,752,752,752,752,752,752,752,641,692,117,1,0,0,0,0,0,0
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN
min,EMGO,Emperor Goose - EMGO,Acanthis cabaret,Emperor Goose,TC000104,empgoo,10057.0,ABBA,ACCA,ABFL,True,NaN,NaN,NaN,NaN,NaN,NaN
25%,SDKI,Sula Dwarf-Kingfisher - SDKI,Conirostrum binghami,Sula Dwarf-Kingfisher,TC013509,vardwk2,17264.0,CHTA,CONE,DMAN,True,NaN,NaN,NaN,NaN,NaN,NaN
50%,subbus1,Sulphur-breasted Bushshrike - subbus1,Mascarinus barklyi,Sulphur-breasted Bushshrike,TC007087,subbus1,23377.012993,MAMA MAMU,MASI,LERE,True,NaN,NaN,NaN,NaN,NaN,NaN


,shorthand,longhand,sci_name,com_name_x,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,ebird,com_name_y,extinct,n_recs,n_bg_recs,species_index,xc
10461,BCQU,Black-chinned Quailfinch - BCQU,Ortygospiza gabonensis,Black-chinned Quailfinch,TC012115,rebqua1,31941.0,BCQU QUAI,ORGA,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10462,AFQU,African Quailfinch - AFQU,Ortygospiza fuscocrissa,African Quailfinch,TC012116,afrqua2,31945.0,AFQU QUAI,ORFU,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10491,GHSI,Gray-headed Silverbill - GHSI,Odontospiza griseicapilla,Gray-headed Silverbill,TC012149,gyhsil1,32026.0,GHSI,ODGR,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10492,BRMA,Bronze Mannikin - BRMA,Spermestes cucullata,Bronze Mannikin,TC012150,broman1,32027.0,NaN,SPCU,BRMA,True,NaN,NaN,NaN,NaN,NaN,NaN
10493,bawman1,Black-and-white Mannikin - bawman1,Spermestes bicolor,Black-and-white Mannikin,TC012151,bawman1,32030.0,BAWB BAWM,SPBI,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10494,MAMA,Magpie Mannikin - MAMA,Spermestes fringilloides,Magpie Mannikin,TC012154,magman1,32037.0,MAMA,SPFR,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10497,madmun1,Madagascar Munia - madmun1,Lonchura nana,Madagascar Munia,TC012157,madmun1,32042.0,MAMA MAMU,LONA,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10519,NHMU,New Hanover Munia - NHMU,Lonchura nigerrima,New Hanover Munia,TC012179,nehmun1,32127.0,NHMU HUMA,LONI,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN
10550,_UNK,Unknown - _UNK,Unknown,Unknown,TC___UNK,___unk,_UNK,_UNK,_UNK,_UNK,True,NaN,NaN,NaN,NaN,NaN,NaN
10551,_NON,No species - _NON,No species,No species,TC___NON,___non,_NON,_NON,_NON,_NON,True,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# xc only:
#   - 911
#   - None have com_name
xc_only = outer_join[lambda df: pd.isnull(df.ebird) & pd.notnull(df.xc)]
display(
    df_summary(xc_only),
    xc_only[-10:],
)

,shorthand,longhand,sci_name,com_name_x,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,ebird,com_name_y,extinct,n_recs,n_bg_recs,species_index,xc
dtype,category,category,object,category,category,category,object,object,object,object,object,object,object,float64,float64,float64,object
sizeof,35.6 KiB,35.6 KiB,75.1 KiB,35.6 KiB,35.6 KiB,35.6 KiB,35.6 KiB,35.6 KiB,35.6 KiB,35.6 KiB,35.6 KiB,75.0 KiB,38.2 KiB,35.6 KiB,35.6 KiB,35.6 KiB,39.2 KiB
len,911,911,911,911,911,911,911,911,911,911,911,911,911,911,911,911,911
count,0,0,911,0,0,0,0,0,0,0,0,911,42,911,911,911,911
nunique,0,0,911,0,0,0,0,0,0,0,0,911,1,119,85,911,1
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.352,13.712,5818.847,1.000
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35.301,58.684,3034.276,0.000
min,_NON,No species - _NON,Accipiter chilensis,No species,TC___NON,___non,NaN,NaN,NaN,NaN,NaN,Abbott's Babbler,Extinct,0.0,0.0,112.0,True
25%,_NON,No species - _NON,Diaphorapteryx hawkinsi,No species,TC___NON,___non,NaN,NaN,NaN,NaN,NaN,Choco Warbler,Extinct,3.0,0.0,3230.0,True
50%,_NON,No species - _NON,Macronus gularis,No species,TC___NON,___non,NaN,NaN,NaN,NaN,NaN,Malagasy Turtle Dove,Extinct,12.0,1.0,5770.0,True


,shorthand,longhand,sci_name,com_name_x,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,ebird,com_name_y,extinct,n_recs,n_bg_recs,species_index,xc
11453,nan,nan,Pyrgilauda theresae,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Afghan Snowfinch,NaN,0.0,0.0,10600.0,True
11454,nan,nan,Foudia aldabrana,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Aldabra Fody,NaN,0.0,0.0,10600.0,True
11455,nan,nan,Coccopygia bocagei,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Angolan Waxbill,NaN,0.0,0.0,10600.0,True
11456,nan,nan,Lonchura leucosticta,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,White-spotted Mannikin,NaN,0.0,0.0,10600.0,True
11457,nan,nan,Macronyx sharpei,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Sharpe's Longclaw,NaN,0.0,0.0,10700.0,True
11458,nan,nan,Leucosticte sillemi,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Sillem's Mountain Finch,NaN,0.0,0.0,10700.0,True
11459,nan,nan,Crithagra canicapilla,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,West African Seedeater,NaN,0.0,0.0,10700.0,True
11460,nan,nan,Spinus cucullata,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Red Siskin,NaN,0.0,0.0,10700.0,True
11461,nan,nan,Pipilo naufragus,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Bermuda Towhee,Extinct,0.0,0.0,10700.0,True
11462,nan,nan,Hemispingus goeringi,nan,nan,nan,NaN,NaN,NaN,NaN,NaN,Slaty-backed Hemispingus,NaN,0.0,0.0,10700.0,True


In [ ]:
species_xc_counts = (
    pd.merge(
        metadata.species.df,
        xc_counts.drop(columns=['com_name']),
        how='left',
        on='sci_name',
    )
    [['species_code', 'n_recs', 'n_bg_recs']]
    .rename(columns={
        'n_recs': 'n_xc_recs',
        'n_bg_recs': 'n_xc_bg_recs',
    })
)
display(
    df_summary(species_xc_counts),
    species_xc_counts[-10:],
)

,species_code,n_xc_recs,n_xc_bg_recs
dtype,category,float64,float64
sizeof,741.4 KiB,412.2 KiB,412.2 KiB
len,10552,10552,10552
count,10552,9800,9800
nunique,10552,393,422
mean,NaN,37.592,31.933
std,NaN,104.832,207.251
min,ostric2,0.0,0.0
25%,satspi1,5.0,0.0
50%,vebtyr1,14.0,1.0


,species_code,n_xc_recs,n_xc_bg_recs
10542,jopind1,0.0,0.0
10543,jamind1,1.0,0.0
10544,bakind1,1.0,0.0
10545,camind1,0.0,0.0
10546,varind1,11.0,0.0
10547,purind1,8.0,0.0
10548,greind1,2.0,0.0
10549,parwea1,5.0,0.0
10550,___unk,NaN,NaN
10551,___non,NaN,NaN


In [ ]:
display(
    df_summary(metadata.xc_counts.df),
    metadata.xc_counts.df[-10:],
)

,species_code,n_xc_recs,n_xc_bg_recs
dtype,category,float64,float64
sizeof,741.4 KiB,412.2 KiB,412.2 KiB
len,10552,10552,10552
count,10552,9800,9800
nunique,10552,393,422
mean,NaN,37.592,31.933
std,NaN,104.832,207.251
min,ostric2,0.0,0.0
25%,satspi1,5.0,0.0
50%,vebtyr1,14.0,1.0


,species_code,n_xc_recs,n_xc_bg_recs
10542,jopind1,0.0,0.0
10543,jamind1,1.0,0.0
10544,bakind1,1.0,0.0
10545,camind1,0.0,0.0
10546,varind1,11.0,0.0
10547,purind1,8.0,0.0
10548,greind1,2.0,0.0
10549,parwea1,5.0,0.0
10550,___unk,NaN,NaN
10551,___non,NaN,NaN


In [ ]:
species_with_xc_counts = (
    pd.merge(
        metadata.species.df,
        species_xc_counts,
        how='left',
        on='species_code',
    )
)
display(
    df_summary(species_with_xc_counts),
    species_with_xc_counts[-10:],
)

,shorthand,longhand,sci_name,com_name,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,n_xc_recs,n_xc_bg_recs
dtype,category,category,category,category,category,category,object,object,object,object,float64,float64
sizeof,719.2 KiB,943.4 KiB,869.0 KiB,863.5 KiB,752.9 KiB,741.4 KiB,743.2 KiB,680.3 KiB,711.4 KiB,473.1 KiB,412.2 KiB,412.2 KiB
len,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552
count,10552,10552,10552,10552,10552,10552,10552,9038,10552,2144,9800,9800
nunique,10552,10552,10552,10552,10552,10552,10552,7090,6250,2144,393,422
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.592,31.933
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.832,207.251
min,COOS,Common Ostrich - COOS,Struthio camelus,Common Ostrich,TC000002,ostric2,10000.0,AAKS,ABAB,ABDU,0.0,0.0
25%,STSP,Sao Tome Spinetail - STSP,Zoonavena thomensis,Sao Tome Spinetail,TC003784,satspi1,15963.0,CRCH CARC,COSQ,CPOT,5.0,0.0
50%,VEBT,Velvety Black-Tyrant - VEBT,Knipolegus nigerrimus,Velvety Black-Tyrant,TC006272,vebtyr1,23377.01045,MARI,LOSU,LIBI,14.0,1.0


,shorthand,longhand,sci_name,com_name,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,n_xc_recs,n_xc_bg_recs
10542,JPIN,Jos Plateau Indigobird - JPIN,Vidua maryae,Jos Plateau Indigobird,TC012204,jopind1,32168.0,JPIN,VIMA,NaN,0.0,0.0
10543,JAIN,Jambandu Indigobird - JAIN,Vidua raricola,Jambandu Indigobird,TC012205,jamind1,32169.0,JAIN,VIRA,NaN,1.0,0.0
10544,BAIN,Baka Indigobird - BAIN,Vidua larvaticola,Baka Indigobird,TC012206,bakind1,32170.0,BAIN,VILA,NaN,1.0,0.0
10545,CAIN,Cameroon Indigobird - CAIN,Vidua camerunensis,Cameroon Indigobird,TC012207,camind1,32171.0,CAIN,VICA,NaN,0.0,0.0
10546,VAIN,Variable Indigobird - VAIN,Vidua funerea,Variable Indigobird,TC012208,varind1,32172.0,VAIN DUIN,VIFU,NaN,11.0,0.0
10547,PUIN,Purple Indigobird - PUIN,Vidua purpurascens,Purple Indigobird,TC012209,purind1,32175.0,PUIN,VIPU,NaN,8.0,0.0
10548,GRIN,Green Indigobird - GRIN,Vidua codringtoni,Green Indigobird,TC012210,greind1,32176.0,GRIN ZAIN,VICO,NaN,2.0,0.0
10549,PAWE,Parasitic Weaver - PAWE,Anomalospiza imberbis,Parasitic Weaver,TC012212,parwea1,32178.0,PAWE CUFI,ANIM,NaN,5.0,0.0
10550,_UNK,Unknown - _UNK,Unknown,Unknown,TC___UNK,___unk,_UNK,_UNK,_UNK,_UNK,NaN,NaN
10551,_NON,No species - _NON,No species,No species,TC___NON,___non,_NON,_NON,_NON,_NON,NaN,NaN


In [ ]:
display(
    df_summary(metadata.xc_counts.with_species),
    metadata.xc_counts.with_species[-10:],
)

,shorthand,longhand,sci_name,com_name,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,n_xc_recs,n_xc_bg_recs
dtype,category,category,category,category,category,category,object,object,object,object,float64,float64
sizeof,719.2 KiB,943.4 KiB,869.0 KiB,863.5 KiB,752.9 KiB,741.4 KiB,743.2 KiB,680.3 KiB,711.4 KiB,473.1 KiB,412.2 KiB,412.2 KiB
len,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552,10552
count,10552,10552,10552,10552,10552,10552,10552,9038,10552,2144,9800,9800
nunique,10552,10552,10552,10552,10552,10552,10552,7090,6250,2144,393,422
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,37.592,31.933
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,104.832,207.251
min,COOS,Common Ostrich - COOS,Struthio camelus,Common Ostrich,TC000002,ostric2,10000.0,AAKS,ABAB,ABDU,0.0,0.0
25%,STSP,Sao Tome Spinetail - STSP,Zoonavena thomensis,Sao Tome Spinetail,TC003784,satspi1,15963.0,CRCH CARC,COSQ,CPOT,5.0,0.0
50%,VEBT,Velvety Black-Tyrant - VEBT,Knipolegus nigerrimus,Velvety Black-Tyrant,TC006272,vebtyr1,23377.01045,MARI,LOSU,LIBI,14.0,1.0


,shorthand,longhand,sci_name,com_name,taxon_id,species_code,taxon_order,com_name_codes,sci_name_codes,banding_codes,n_xc_recs,n_xc_bg_recs
10542,JPIN,Jos Plateau Indigobird - JPIN,Vidua maryae,Jos Plateau Indigobird,TC012204,jopind1,32168.0,JPIN,VIMA,NaN,0.0,0.0
10543,JAIN,Jambandu Indigobird - JAIN,Vidua raricola,Jambandu Indigobird,TC012205,jamind1,32169.0,JAIN,VIRA,NaN,1.0,0.0
10544,BAIN,Baka Indigobird - BAIN,Vidua larvaticola,Baka Indigobird,TC012206,bakind1,32170.0,BAIN,VILA,NaN,1.0,0.0
10545,CAIN,Cameroon Indigobird - CAIN,Vidua camerunensis,Cameroon Indigobird,TC012207,camind1,32171.0,CAIN,VICA,NaN,0.0,0.0
10546,VAIN,Variable Indigobird - VAIN,Vidua funerea,Variable Indigobird,TC012208,varind1,32172.0,VAIN DUIN,VIFU,NaN,11.0,0.0
10547,PUIN,Purple Indigobird - PUIN,Vidua purpurascens,Purple Indigobird,TC012209,purind1,32175.0,PUIN,VIPU,NaN,8.0,0.0
10548,GRIN,Green Indigobird - GRIN,Vidua codringtoni,Green Indigobird,TC012210,greind1,32176.0,GRIN ZAIN,VICO,NaN,2.0,0.0
10549,PAWE,Parasitic Weaver - PAWE,Anomalospiza imberbis,Parasitic Weaver,TC012212,parwea1,32178.0,PAWE CUFI,ANIM,NaN,5.0,0.0
10550,_UNK,Unknown - _UNK,Unknown,Unknown,TC___UNK,___unk,_UNK,_UNK,_UNK,_UNK,NaN,NaN
10551,_NON,No species - _NON,No species,No species,TC___NON,___non,_NON,_NON,_NON,_NON,NaN,NaN


In [ ]:
# FIXME Join on com_name after sci_name, to catch sci_names that don't match, e.g.
#   - xc_counts: com_name='Orange-crowned Warbler', sci_name='Leiothlypis celata'
#   - species:   com_name='Orange-crowned Warbler', sci_name='Oreothlypis celata'
display(
    metadata.species.df[lambda df: df.com_name == 'Orange-crowned Warbler'].T,
    metadata.xc_counts._raw_df[lambda df: df.com_name == 'Orange-crowned Warbler'].T,
)

,9189
shorthand,OCWA
longhand,Orange-crowned Warbler - OCWA
sci_name,Oreothlypis celata
com_name,Orange-crowned Warbler
taxon_id,TC010478
species_code,orcwar
taxon_order,27793.0
com_name_codes,NaN
sci_name_codes,ORCE
banding_codes,OCWA


,246
com_name,Orange-crowned Warbler
sci_name,Leiothlypis celata
extinct,NaN
n_recs,214
n_bg_recs,206
species_index,247
